In [1]:
import pandas as pd
from framework.storages.database.ChromaDB import ChromaDB
from framework.models.language_models.IBMModel import IBMModel

In [13]:
print(df['Content'][0])

The document contains the following text:
BỘ GIÁO DỤC VÀ ĐÀO TẠO
Số: 29/2023/TT-BGDĐT
ĐẠI HỌC QUỐC GIA TP.HCM
Số đến:
CÔNG VĂN ĐẾN
Ngày đến:
211.
11/01/2024
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập – Tự do - Hạnh phúc
Hà Nội, ngày 29 tháng 12 năm 2023
THÔNG TƯ
Quy định chi tiết thi hành một số điều của Luật Thi đua, khen thưởng
đối với ngành Giáo dục


In [14]:
print(df['Content'][1])

đối với ngành Giáo dục
Căn cứ Luật Thi đua, khen thưởng ngày 15 tháng 6 năm 2022;
Căn cứ Nghị định số 86/2022/NĐ-CP ngày 24 tháng 10 năm 2022 của
Chính phủ quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ
Giáo dục và Đào tạo;
Theo đề nghị của Vụ trưởng Vụ Tổ chức cán bộ;
Bộ trưởng Bộ Giáo dục và Đào tạo ban hành Thông tư quy định chi tiết
thi hành một số


In [15]:
print(df['Content'][2])

thi hành một số
điều của Luật Thi đua, khen thưởng đối với ngành Giáo dục.
Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng
1. Thông tư này quy định chi tiết thi hành khoản 4 Điều 24, khoản 3 Điều
26, khoản 3 Điều 27, khoản 3 Điều 28, khoản 3 Điều 32, khoản 2 Điều 71,
khoản 6 Điều 74, khoản 2 Điều 75, khoản 3 và khoản 4 Điều 76, khoản 6 Điều
84 của Luật Thi đua, khen thưởng.


In [16]:
db = ChromaDB()
collection = db.create_collection('test')

In [17]:
df = pd.read_csv("documentation/TT 29 TDKT.csv")
# Truy xuất metadata theo đúng format
## TODO: Sau này nên có hàm hỗ trợ việc này (utils)
metas = [
    {
        "Title": val[0],
        "Chunk ID": val[1]
    }
    for val in df.values
]

In [18]:
db.add_documents(collection, documents=list(df['Content'].values), metadatas=metas)
print("Đã đọc tài liệu + Tạo vector database in-memory (ChromaDB)")
print("Kho dữ liệu có", collection.count(), "dòng")

Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
Insert of existing embedding ID: 28
I

Đã đọc tài liệu + Tạo vector database in-memory (ChromaDB)
Kho dữ liệu có 78 dòng


In [19]:
model_id = "mistralai/mistral-large"
apikey = "XY6YolsO_sW5fIzdKlQyZqUNt0uBfylcWrxxJ46UIhIw"
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": apikey
}
parameters = {
    "decoding_method": "greedy", #Đổi sang sampling thì mô hình sẽ "sáng tạo" hơn
    "max_new_tokens": 300, #Chiều dài tối đa cho output - để đề phòng mô hình nói nhiều
    "repetition_penalty": 1 #Giá trị từ 1-2 (1 thì mô hình luôn trả lời y chang, 2 thì ngược lại)
}
project_id = "b2ce1b2d-0641-4d0e-95cb-39f7dca76da1"

model = IBMModel(model_id, credentials, project_id, parameters)
print("Khởi tạo mô hình thành công")

Khởi tạo mô hình thành công


In [20]:
query = "Đối tượng áp dụng thông tư"


results = db.search(collection, query, n_results=3)
print("Tìm kiếm trong kho dữ liệu thành công")
context = ""
results = results['documents'][0]
for r in results:
    context = context + r + '\n\n'
prompt = f"""Bạn là một trợ lý ảo. Hãy trả lời câu hỏi của người dùng dựa trên ngữ cảnh được cung cấp bên dưới. Hãy trả lời bằng tiếng Việt. Nếu bạn không thể trả lời câu hỏi với ngữ cảnh được cho, hãy trả lời tôi không biết.
Tuyệt đối không đưa thông tin sai sự thật. Luôn trả lời bằng giọng điệu tích cực và không chứa hàm ý tấn công đến bất kỳ một cá nhân chủng tộc nào.

NGỮ CẢNH:
{context}

CÂU HỎI: {query}
"""

Tìm kiếm trong kho dữ liệu thành công


In [21]:
print(prompt)

Bạn là một trợ lý ảo. Hãy trả lời câu hỏi của người dùng dựa trên ngữ cảnh được cung cấp bên dưới. Hãy trả lời bằng tiếng Việt. Nếu bạn không thể trả lời câu hỏi với ngữ cảnh được cho, hãy trả lời tôi không biết.
Tuyệt đối không đưa thông tin sai sự thật. Luôn trả lời bằng giọng điệu tích cực và không chứa hàm ý tấn công đến bất kỳ một cá nhân chủng tộc nào.

NGỮ CẢNH:
thưởng của đơn vị trình.
Điều 15. Hiệu lực thi hành và trách nhiệm thi hành
1. Thông tư này có hiệu lực thi hành kể từ ngày 15 tháng 02 năm 2024
và thay thế Thông tư số 21/2020/TT-BGDĐT ngày 31 tháng 7 năm 2020 của
Bộ trưởng Bộ Giáo dục và Đào tạo hướng dẫn công tác thi đua, khen thưởng
ngành Giáo dục.
Urew
11
2. Trường hợp các văn bản quy phạm pháp luật được dẫn chiếu tại Thông

84 của Luật Thi đua, khen thưởng.
2. Thông tư này áp dụng đối với tập thể, cán bộ, công chức, viên chức,
người lao động trong ngành Giáo dục; người học; tập thể, hộ gia đình, cá nhân
người Việt Nam, người Việt Nam định cư ở nước ngoài; tập thể v

In [22]:
response = model.generate(prompt)
print("Câu hỏi query:", query)
print("---------------")
print(response)

Câu hỏi query: Đối tượng áp dụng thông tư
---------------

TRẢ LỜI: Thông tư này áp dụng đối với tập thể, cán bộ, công chức, viên chức, người lao động trong ngành Giáo dục; người học; tập thể, hộ gia đình, cá nhân người Việt Nam, người Việt Nam định cư ở nước ngoài; tập thể và người nước ngoài có thành tích đóng góp cho sự nghiệp xây dựng và phát triển giáo dục, đào tạo.
